# Data Preprocessing

## Import libraries

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/output/output_dataset.csv')

df.head()

,code,url,titre,contenu,langue,nb_caracteres,nb_mots,date_publication,lieu,maladie,animal,source_publication,resume_50_mots,resume_100_mots,resume_150_mots
0,code151,https://lc.cx/nKVbsM,Bluetongue virus was detected in Europe,Tridge LogoGlobal trading platform Tridge Get ...,en,3891,617,16/10/2023,Allemagne,Bluetongue,"Ovins, Bovins",médias,Le virus de la fièvre catarrhale ovine (Blueto...,Le virus de la fièvre catarrhale ovine (Blueto...,Le virus de la fièvre catarrhale ovine (Blueto...
1,code152,https://lc.cx/sXWRhi,Bluetongue officially confirmed in Germany,Tridge LogoGlobal trading platform Tridge Get ...,en,3942,610,17 octobre 2023,Allemagne,Bluetongue,"Bovins, Ovins",médias,Un animal bovin a été affecté par la fièvre ca...,La maladie a été détectée après les Pays-Bas e...,Le virus de la fièvre catarrhale ovine (blueto...
2,code153,https://www.agriland.ie/farming-news/daera-urg...,DAERA urges farmers to be vigilant for blueton...,Latest News Beef Dairy Sheep Tillage Machinery...,en,4934,770,18 octobre 2023,Irlande du Nord,Bluetongue,"Bovins, Ovins",médias,Le DAERA de l'Irlande du Nord a mis en garde l...,Le DAERA de l'Irlande du Nord a mis en garde l...,Le DAERA de l'Irlande du Nord a mis en garde l...
3,code154,https://lc.cx/fnYicq,Erreur,Erreur: Message: unknown error: net::ERR_NAME_...,en,566,42,19 octobre 2023,Allemagne,Bluetongue,"Ovins, Bovins",médias,Le virus de la fièvre catarrhale ovine (Blueto...,Le virus de la fièvre catarrhale ovine (Blueto...,Le virus de la fièvre catarrhale ovine (Blueto...
4,code155,https://lc.cx/V_Nr3n,اقتصاد,بعد موجة بق الفراش، ترزح العديد من الدول الأور...,ar,1169,191,19 octobre 2023,France,Bluetongue,"Ovins, Bovins",médias,La France détecte des cas de Bluetongue chez l...,Le virus de la fièvre catarrhale ovine (Blueto...,La France a signalé une nouvelle épidémie de B...


In [3]:
print(df['maladie'])

0                                            Bluetongue
1                                            Bluetongue
2                                            Bluetongue
3                                            Bluetongue
4                                            Bluetongue
5                                            Bluetongue
6                                            Bluetongue
7                  Fièvre catarrhale ovine (Bluetongue)
8                       Dermatose nodulaire contagieuse
9                      Maladie hémorragique épizootique
10               Maladie hémorragique épizootique (MHE)
11                     Maladie hémorragique épizootique
12                      Dermatose nodulaire contagieuse
13                                          Tuberculose
14                      Dermatose nodulaire contagieuse
15                                           Bluetongue
16                     Maladie hémorragique épizootique
17                                             i

In [6]:
import re
from difflib import SequenceMatcher
disease_mapping = {
    # Bluetongue / Fièvre catarrhale ovine
    'bluetongue': 'Fièvre catarrhale ovine (Bluetongue)',
    'fièvre catarrhale ovine': 'Fièvre catarrhale ovine (Bluetongue)',
    'fco': 'Fièvre catarrhale ovine (Bluetongue)',
    'langue bleue': 'Fièvre catarrhale ovine (Bluetongue)',
    'bluetongue (inf. par le virus de la)': 'Fièvre catarrhale ovine (Bluetongue)',
    
    # Maladie hémorragique épizootique
    'maladie hémorragique épizootique': 'Maladie hémorragique épizootique (MHE)',
    'mhe': 'Maladie hémorragique épizootique (MHE)',
    'maladie hémorragique épizootique (inf. par le virus de la)': 'Maladie hémorragique épizootique (MHE)',
    
    # Dermatose nodulaire contagieuse
    'dermatose nodulaire contagieuse': 'Dermatose nodulaire contagieuse',
    'dnc': 'Dermatose nodulaire contagieuse',
    'dermatose nodulaire contagieuse (inf. par le virus de la)': 'Dermatose nodulaire contagieuse',
    
    # West Nile / Fièvre du Nil occidental
    'fièvre de west nile': 'Fièvre du Nil occidental (West Nile)',
    'west nile virus': 'Fièvre du Nil occidental (West Nile)',
    'wnv': 'Fièvre du Nil occidental (West Nile)',
    'virus du nil occidental': 'Fièvre du Nil occidental (West Nile)',
    
    # Influenza équine
    'grippe équine': 'Influenza équine',
    'grippe équine (inf. par le virus de la)': 'Influenza équine',
    'equine influenza': 'Influenza équine',
    
    # SARS-CoV-2
    'sars-cov-2': 'SARS-CoV-2 chez les animaux',
    'sars-cov-2 chez les animaux': 'SARS-CoV-2 chez les animaux',
    'sars-cov-2 chez les animaux (inf. par le)': 'SARS-CoV-2 chez les animaux',
    
    # Autres maladies
    'tuberculose': 'Tuberculose bovine',
    'clavelée et variole caprine': 'Clavelée et variole caprine',
    'nécrose hématopoïétique infectieuse': 'Nécrose hématopoïétique infectieuse',
    'aethina tumida': 'Petit coléoptère des ruches (Aethina tumida)',
    'ppr': 'Peste des Petits Ruminants (PPR)',
    'ppcb': 'Péripneumonie Contagieuse Bovine (PPCB)',
}

def normalize_disease_name(disease_name):
    """
    Normalise le nom d'une maladie
    """
    if pd.isna(disease_name):
        return 'Maladie non spécifiée'
    
    # Convertir en minuscules et nettoyer
    disease_clean = str(disease_name).lower().strip()
    
    # Chercher dans le dictionnaire
    if disease_clean in disease_mapping:
        return disease_mapping[disease_clean]
    
    # Chercher une correspondance partielle
    for key, value in disease_mapping.items():
        if key in disease_clean or disease_clean in key:
            return value
    
    # Si aucune correspondance, retourner le nom original capitalisé
    return disease_name.strip().title()

# Appliquer la normalisation
df['maladie_normalisee'] = df['maladie'].apply(normalize_disease_name)

# Afficher les résultats
print("=== Statistiques avant normalisation ===")
print(f"Nombre de maladies uniques : {df['maladie'].nunique()}")
print("\nDistribution des maladies :")
print(df['maladie'].value_counts())

print("\n=== Statistiques après normalisation ===")
print(f"Nombre de maladies uniques : {df['maladie_normalisee'].nunique()}")
print("\nDistribution des maladies normalisées :")
print(df['maladie_normalisee'].value_counts())

# Afficher un tableau de correspondance
print("\n=== Tableau de correspondance ===")
correspondance = df[['maladie', 'maladie_normalisee']].drop_duplicates().sort_values('maladie_normalisee')
print(correspondance)

# Sauvegarder le résultat
df.to_csv('donnees_normalisees.csv', index=False)
print("\n✅ Fichier sauvegardé : donnees_normalisees.csv")

# Fonction optionnelle pour ajouter de nouvelles correspondances
def add_disease_mapping(original, normalized):
    """
    Ajoute une nouvelle correspondance au dictionnaire
    """
    disease_mapping[original.lower().strip()] = normalized
    print(f"✅ Ajouté : '{original}' -> '{normalized}'")


=== Statistiques avant normalisation ===
Nombre de maladies uniques : 20

Distribution des maladies :
maladie
Bluetongue                                                                     15
Dermatose nodulaire contagieuse                                                 7
Maladie hémorragique épizootique                                                6
Fièvre de West Nile                                                             3
inconnue                                                                        2
SARS-CoV-2 chez les animaux (Inf. par le)                                       2
Fièvre catarrhale ovine (Bluetongue)                                            2
Clavelée et variole caprine                                                     1
Infection à virus West Nile                                                     1
Fièvre hémorragique de Crimée-Congo                                             1
Nécrose hématopoïétique infectieuse (Inf. par le virus de la)         

In [7]:
df.head()

,code,url,titre,contenu,langue,nb_caracteres,nb_mots,date_publication,lieu,maladie,animal,source_publication,resume_50_mots,resume_100_mots,resume_150_mots,maladie_normalisee
0,code151,https://lc.cx/nKVbsM,Bluetongue virus was detected in Europe,Tridge LogoGlobal trading platform Tridge Get ...,en,3891,617,16/10/2023,Allemagne,Bluetongue,"Ovins, Bovins",médias,Le virus de la fièvre catarrhale ovine (Blueto...,Le virus de la fièvre catarrhale ovine (Blueto...,Le virus de la fièvre catarrhale ovine (Blueto...,Fièvre catarrhale ovine (Bluetongue)
1,code152,https://lc.cx/sXWRhi,Bluetongue officially confirmed in Germany,Tridge LogoGlobal trading platform Tridge Get ...,en,3942,610,17 octobre 2023,Allemagne,Bluetongue,"Bovins, Ovins",médias,Un animal bovin a été affecté par la fièvre ca...,La maladie a été détectée après les Pays-Bas e...,Le virus de la fièvre catarrhale ovine (blueto...,Fièvre catarrhale ovine (Bluetongue)
2,code153,https://www.agriland.ie/farming-news/daera-urg...,DAERA urges farmers to be vigilant for blueton...,Latest News Beef Dairy Sheep Tillage Machinery...,en,4934,770,18 octobre 2023,Irlande du Nord,Bluetongue,"Bovins, Ovins",médias,Le DAERA de l'Irlande du Nord a mis en garde l...,Le DAERA de l'Irlande du Nord a mis en garde l...,Le DAERA de l'Irlande du Nord a mis en garde l...,Fièvre catarrhale ovine (Bluetongue)
3,code154,https://lc.cx/fnYicq,Erreur,Erreur: Message: unknown error: net::ERR_NAME_...,en,566,42,19 octobre 2023,Allemagne,Bluetongue,"Ovins, Bovins",médias,Le virus de la fièvre catarrhale ovine (Blueto...,Le virus de la fièvre catarrhale ovine (Blueto...,Le virus de la fièvre catarrhale ovine (Blueto...,Fièvre catarrhale ovine (Bluetongue)
4,code155,https://lc.cx/V_Nr3n,اقتصاد,بعد موجة بق الفراش، ترزح العديد من الدول الأور...,ar,1169,191,19 octobre 2023,France,Bluetongue,"Ovins, Bovins",médias,La France détecte des cas de Bluetongue chez l...,Le virus de la fièvre catarrhale ovine (Blueto...,La France a signalé une nouvelle épidémie de B...,Fièvre catarrhale ovine (Bluetongue)


In [8]:
df_clean = df[
    ~df['titre'].str.contains('page non trouvée|404|erreur|not found', case=False, na=False) &
    ~df['maladie'].str.contains('inconnu', case=False, na=False) &
    df['maladie'].notna()
]

print(f"Lignes après nettoyage : {len(df_clean)}")
print(f"Lignes supprimées : {len(df) - len(df_clean)}")

# Sauvegarder
df_clean.to_csv('dataset_nettoye.csv', index=False)
print("✅ Dataset nettoyé sauvegardé !")

Lignes après nettoyage : 43
Lignes supprimées : 7
✅ Dataset nettoyé sauvegardé !


In [7]:
df.columns

Index(['code', 'url', 'titre', 'contenu', 'langue', 'nb_caracteres', 'nb_mots',
       'date_publication', 'lieu', 'maladie', 'animal', 'source_publication',
       'resume_50_mots', 'resume_100_mots', 'resume_150_mots'],
      dtype='object')

In [ ]:
from deep_translator import GoogleTranslator
from googletrans import Translator
from textblob import TextBlob
import pandas as pd
import re

# Colonnes à traduire
cols = ['titre', 'contenu', 'lieu', 'maladie', 'animal',
        'resume_50_mots', 'resume_100_mots', 'resume_150_mots']

dt = GoogleTranslator(source="auto", target="fr")
gt = Translator()

url_pattern = re.compile(r'^https?://')

def fallback_translate(text):
    try:
        return gt.translate(text, dest='fr').text
    except:
        try:
            return str(TextBlob(text).translate(to='fr'))
        except:
            return text

def translate_text(text):
    if pd.isna(text) or not isinstance(text, str):
        return text
    if url_pattern.match(text.strip()):
        return text

    # 1) deep_translator (rapide)
    try:
        return dt.translate(text)
    except:
        print("⚠️ deep_translator failed → using fallback")
        return fallback_translate(text)

# Traduire uniquement la colonne contenu
df['contenu'] = df['contenu'].apply(translate_text)

print("✅ Traduction terminée")


⚠️ deep_translator failed → using fallback


C:\Users\rania\AppData\Local\Temp\ipykernel_8252\2113286045.py:18: RuntimeWarning: coroutine 'Translator.translate' was never awaited
  return gt.translate(text, dest='fr').text


⚠️ deep_translator failed → using fallback
⚠️ deep_translator failed → using fallback
⚠️ deep_translator failed → using fallback
⚠️ deep_translator failed → using fallback
⚠️ deep_translator failed → using fallback
⚠️ deep_translator failed → using fallback
⚠️ deep_translator failed → using fallback
⚠️ deep_translator failed → using fallback
⚠️ deep_translator failed → using fallback
⚠️ deep_translator failed → using fallback
⚠️ deep_translator failed → using fallback
⚠️ deep_translator failed → using fallback
✅ Traduction terminée. Les codes 178 et 192 ont été traduits correctement.


In [9]:
df.head()

,code,url,titre,contenu,langue,nb_caracteres,nb_mots,date_publication,lieu,maladie,animal,source_publication,resume_50_mots,resume_100_mots,resume_150_mots
0,code151,https://lc.cx/nKVbsM,Bluetongue virus was detected in Europe,Tridge LogoPlateforme de trading mondiale Trid...,en,3949,624,16/10/2023,Allemagne,Maladie hémorragique épizootique,"Ovins, Bovins",médias,La maladie hémorragique épizootique a été déte...,Le virus a été identifié sur une ferme de mout...,La maladie hémorragique épizootique a été déte...
1,code152,https://lc.cx/sXWRhi,Bluetongue officially confirmed in Germany,Logo TridgePlateforme commerciale mondiale Tri...,en,4000,617,17/10/2023,Allemagne,Fièvre catarrhale ovine,"Bovins, Ovins",médias,Une maladie a été détectée sur une ferme en Al...,Un animal bovin a contracté le virus de la blu...,"L'Allemagne, la Belgique et les Pays-Bas discu..."
2,code153,https://www.agriland.ie/farming-news/daera-urg...,DAERA urges farmers to be vigilant for blueton...,Dernières nouvelles Boeuf Laitier Mouton Machi...,en,4947,770,19/10/2023,Irlande du Nord,"Bluetongue, EHD","Bovins, Ovins, Cervidés",médias,Le DAERA alerte les agriculteurs irlandais à l...,La France a confirmé l'apparition de la maladi...,Le DAERA met en place des contrôles stricts su...
3,code154,https://lc.cx/fnYicq,Erreur,Erreur : Message : erreur inconnue : net::ERR_...,en,566,42,inconnue,Maroc,Maladie hémorragique épizootique,"Bovins, Ovins",médias,Le Maroc surveille l'apparition de la maladie ...,L'ONSA met en place des contrôles stricts sur ...,La maladie hémorragique épizootique est une me...
4,code155,https://lc.cx/V_Nr3n,اقتصاد,"Après la vague de punaises de lit, de nombreux...",ar,1169,191,19/10/2023,France,Fièvre catarrhale ovine,"Ovins, Bovins, Équidés",médias,Francia détecte une épidémie de bluetongue che...,La France a signalé une épidémie de bluetongue...,L'épidémie de bluetongue a été détectée en Fra...


In [10]:
# Mots-clés à supprimer
mots_inconnus = ["inconnu", "inconnue", "maladie inconnue"]

# Convertir toutes les colonnes en texte pour faciliter la recherche
df = df.astype(str)

# Supprimer toutes les lignes contenant un des mots-clés
df_clean = df[~df.apply(lambda row: row.str.lower().str.contains('|'.join(mots_inconnus)).any(), axis=1)]

# Sauvegarde
df_clean.to_csv("data/output/output_dataset_cleaned.csv", index=False)

print("OK ! Lignes supprimées :", len(df) - len(df_clean))


OK ! Lignes supprimées : 18


In [11]:
df.head()

,code,url,titre,contenu,langue,nb_caracteres,nb_mots,date_publication,lieu,maladie,animal,source_publication,resume_50_mots,resume_100_mots,resume_150_mots
0,code151,https://lc.cx/nKVbsM,Bluetongue virus was detected in Europe,Tridge LogoPlateforme de trading mondiale Trid...,en,3949,624,16/10/2023,Allemagne,Maladie hémorragique épizootique,"Ovins, Bovins",médias,La maladie hémorragique épizootique a été déte...,Le virus a été identifié sur une ferme de mout...,La maladie hémorragique épizootique a été déte...
1,code152,https://lc.cx/sXWRhi,Bluetongue officially confirmed in Germany,Logo TridgePlateforme commerciale mondiale Tri...,en,4000,617,17/10/2023,Allemagne,Fièvre catarrhale ovine,"Bovins, Ovins",médias,Une maladie a été détectée sur une ferme en Al...,Un animal bovin a contracté le virus de la blu...,"L'Allemagne, la Belgique et les Pays-Bas discu..."
2,code153,https://www.agriland.ie/farming-news/daera-urg...,DAERA urges farmers to be vigilant for blueton...,Dernières nouvelles Boeuf Laitier Mouton Machi...,en,4947,770,19/10/2023,Irlande du Nord,"Bluetongue, EHD","Bovins, Ovins, Cervidés",médias,Le DAERA alerte les agriculteurs irlandais à l...,La France a confirmé l'apparition de la maladi...,Le DAERA met en place des contrôles stricts su...
3,code154,https://lc.cx/fnYicq,Erreur,Erreur : Message : erreur inconnue : net::ERR_...,en,566,42,inconnue,Maroc,Maladie hémorragique épizootique,"Bovins, Ovins",médias,Le Maroc surveille l'apparition de la maladie ...,L'ONSA met en place des contrôles stricts sur ...,La maladie hémorragique épizootique est une me...
4,code155,https://lc.cx/V_Nr3n,اقتصاد,"Après la vague de punaises de lit, de nombreux...",ar,1169,191,19/10/2023,France,Fièvre catarrhale ovine,"Ovins, Bovins, Équidés",médias,Francia détecte une épidémie de bluetongue che...,La France a signalé une épidémie de bluetongue...,L'épidémie de bluetongue a été détectée en Fra...
